# Download dataset from Kaggle

In [0]:
# install kaggle library
!pip install -q kaggle

In [0]:
!mkdir /root/.kaggle

In [0]:
!echo '{"username":"jgupta07","key":"31b5d3936a3059099e6164fdc8b90c82"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d jgupta07/ip102-dataset --force

100% 2.92G/2.94G [00:52<00:01, 12.0MB/s]
100% 2.94G/2.94G [00:53<00:00, 59.5MB/s]


In [5]:
!unzip ip102-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: val/29/23657.jpg        
  inflating: val/29/23664.jpg        
  inflating: val/29/23668.jpg        
  inflating: val/29/23674.jpg        
  inflating: val/29/23687.jpg        
  inflating: val/29/23689.jpg        
  inflating: val/29/23690.jpg        
  inflating: val/29/23699.jpg        
  inflating: val/29/23704.jpg        
  inflating: val/29/23707.jpg        
  inflating: val/29/23712.jpg        
  inflating: val/29/23716.jpg        
  inflating: val/29/23745.jpg        
  inflating: val/29/23746.jpg        
  inflating: val/29/23747.jpg        
  inflating: val/29/23771.jpg        
  inflating: val/29/23792.jpg        
  inflating: val/29/23794.jpg        
  inflating: val/29/23818.jpg        
  inflating: val/29/23820.jpg        
  inflating: val/29/23832.jpg        
  inflating: val/29/23838.jpg        
  inflating: val/29/23839.jpg        
  inflating: val/29/23842.jpg        
  inflating: val/29/23844.jpg        

In [6]:
!ls

classes.txt  ip102-dataset.zip	sample_data  test  train  val


# Load using `keras.preprocessing` 

See [tensorflow docs](https://www.tensorflow.org/tutorials/load_data/images#load_using_keraspreprocessing)

In [7]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

TensorFlow 2.x selected.


In [8]:
# create ImageDataGenerators
datagen_train = keras.preprocessing.image.ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True
)

datagen_test = keras.preprocessing.image.ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True
)

datagen_val = keras.preprocessing.image.ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True
)

# load the data
generator_train = datagen_train.flow_from_directory(
    directory='train/',
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True
)

generator_test = datagen_test.flow_from_directory(
    directory='test/',
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True
)

generator_val = datagen_val.flow_from_directory(
    directory='val/',
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True
)

Found 45095 images belonging to 102 classes.
Found 22619 images belonging to 102 classes.
Found 7508 images belonging to 102 classes.


# Transfer Learning
Fine-tune InceptionV3 on a new set of classes, 
[see here](https://keras.io/applications/)

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K

In [14]:
# create the base pre-trained model
#base_model = InceptionV3(weights='imagenet', include_top=False)
base_model = VGG16(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 102 classes
predictions = Dense(102, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
history1 = model.fit_generator(generator_train)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:19]:
   layer.trainable = False
for layer in model.layers[19:]:
   layer.trainable = True

  ...
    to  
  ['...']
Train for 1410 steps
1410/1410 [==============================] - 297s 210ms/step - loss: 2.7898
0 input_2
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [15]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history2 = model.fit_generator(generator_train, validation_data=generator_val)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1410 steps, validate for 235 steps
1410/1410 [==============================] - 349s 247ms/step - loss: 2.1583 - acc: 0.4636 - val_loss: 2.2660 - val_acc: 0.4401
